# Chapter 4 Training Models

So far we have treated Machine Learning models and their training algorithms mostly like black boxes.If you went through some of the exercises in the previous chapters,you may have been surprised by how much you can get done without knowing anything about what's under the hood:you optimised a regression system,you improved a digit image classifer,and you even built a sapm classifer from scratch,all this without knowing how they actually work.Indeed, in many situations you do not really know the implementation details.

However,having a good understanding of how things work can help you quickly home in on the appropriate model,the right training algorithm to use,and a good set of hyperparameters for your task.Undersatanding what is under the hood will also help you debug issues and perform error analysis more efficiently.Lastly,most of the topics discussed in this chapter will be essential i understanding,building,and training neural networks(discussed in Part 2 of this book).

In this chapter we will start by looking at the Linear Regression model,one of the simplest models there is .We will discuss two very different ways to train it:

* Using a direct "closed-form" equation that directly computes the model parameters that best fit the model to the training set(i.e.,  the model parameters that minimize the cost function over the training set)

* Using an iterative optimization approach called Gradient(CD) that gradually tweaks the model parameters to minimize the cost function over the training set,eventually converging to the same set of paramaters as the first method.We will look at a few variants of Gradient Descent that we will use again and again when we study neural networks in PARAT 2.

Next we will look at Polynomial Regression,a more complex model that can fit non-linear datasets. Since this model has more parameters than Linear Regression,it is more prone to overfitting the training data,so we will look at how to detect whether or not this is the case using learning curves, and then we will look at several regularization techniques that can reduce the risk of overfitting the training set.

Finally,we will look at two more models that are commonly used for classification tasks: Logistic Regression and Softmax Regression.

There will be quite a few math equations in this chapter,using basic notions of linear algebra and calclus. To understand these equations,you will need to know what vectors and matrices are;how to transpose them ,multiply them,and inverse them;and what partial derivatives are.If you are unfamiliar with these concepts,please go through the linear algebra and calculus introductory tutorials available as Jupyter notebooks in the online supplemental material.For those who are truly allergic to mathematics, you should still go through this chapter and simply skip the equations;hopefully, the text will be sufficient to help you understand most of the concepts.

# Linear regression
More generally, a linear model makes a prediction by simply computing a weighted sum of the input features,plus a constant called the bias term(also called the intercept term),as shown in Equation 4-1.

Equation4-1 Linear Regression model prediction
